# High Frequency Trading Algorithm

You have been tasked by the investment firm Renaissance High Frequency Trading (RHFT) to develop an automated trading strategy utilizing a combination of machine learning algorithms and high frequency algorithms. RHFT wants this new algorithm to be based on stock market data of the 30 stocks in the Dow Jones at the minute level and to conduct buys and sells every minute based on 1 min, 5 min, and 10 min Momentum. The CIO asked you to choose the Machine Learning Algorithm best suited for this task and wants you to execute the trades via Alpaca's API.

## Part 1: Prepare the data for training and testing

### Initial Set-Up

In [1]:
import os
from pathlib import Path
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
import datetime
import time
from dotenv import load_dotenv


In [2]:
# Load .env enviroment variables
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_public_key = os.getenv('ALPACA_PUBLIC_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
alpaca_base_url = "https://paper-api.alpaca.markets"

In [4]:
print(f'require {type(alpaca_public_key)} cannot be None')
print(f'require {type(alpaca_secret_key)} cannot be None')

require <class 'str'> cannot be None
require <class 'str'> cannot be None


In [5]:
# Create the Alpaca API object, specifying use of the paper trading account:
alpaca = tradeapi.REST(
    alpaca_public_key,
    alpaca_secret_key,
    base_url=alpaca_base_url,
    api_version="v2")

### Data Generation



#### 1. Create a ticker list, beginning and end dates, and timeframe interval.


In [6]:
# Define a list of tickers
tickers = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL', 'MSFT', 'TSLA']
# declare begin and end date strings
beg_date = '2021-01-05'
end_date = '2021-01-05'
# we convert begin and end date to formats that the ALPACA API requires
start =  pd.Timestamp(f'{beg_date} 09:30:00-0400', tz='America/New_York').replace(hour=9, minute=30, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
end   =  pd.Timestamp(f'{end_date} 16:00:00-0400', tz='America/New_York').replace(hour=16, minute=0, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
# We set the time frequency at which we want to pull prices
timeframe='1Min'

#### 2. Ping the Alpaca API for the data and store it in a DataFrame called `prices` by using the `get_barset` function combined with the `df` method from the Alpaca Trade SDK.

In [7]:
# Pull prices from the ALPACA API
prices = alpaca.get_barset(tickers, timeframe, limit=1000, start=start, end=end).df

In [8]:
prices

AAPL                                      AMZN  \
                              open     high     low    close volume     open   
2021-01-05 09:30:00-05:00  128.960  129.485  128.45  129.485  51887  3166.80   
2021-01-05 09:31:00-05:00  129.480  130.170  129.30  130.060  44188  3173.59   
2021-01-05 09:32:00-05:00  130.170  130.320  129.93  130.020  12852  3175.00   
2021-01-05 09:33:00-05:00  130.090  130.140  129.78  130.120  14192  3181.52   
2021-01-05 09:34:00-05:00  130.150  130.580  130.15  130.510  12002  3183.66   
...                            ...      ...     ...      ...    ...      ...   
2021-01-05 15:56:00-05:00  130.965  130.970  130.84  130.850  12768  3220.70   
2021-01-05 15:57:00-05:00  130.860  131.010  130.86  131.010  17416  3219.95   
2021-01-05 15:58:00-05:00  131.020  131.040  130.86  130.990  28102  3221.78   
2021-01-05 15:59:00-05:00  130.985  131.090  130.95  130.965  26220  3220.14   
2021-01-05 16:00:00-05:00  131.140  131.140  131.14  131.140    100      NaN   

                                                               ...     NFLX  \
                              high      low     close  volume  ...     open   
2021-01-05 09:30:00-05:00  3173.53  3166.80  3172.980  1963.0  ...  521.980   
2021-01-05 09:31:00-05:00  3182.67  3173.58  3177.810  1266.0  ...  520.920   
2021-01-05 09:32:00-05:00  3175.47  3174.91  3175.470   778.0  ...  522.355   
2021-01-05 09:33:00-05:00  3181.52  3177.87  3179.360   660.0  ...  520.840   
2021-01-05 09:34:00-05:00  3189.98  3183.66  3184.015   731.0  ...  521.440   
...                            ...      ...       ...     ...  ...      ...   
2021-01-05 15:56:00-05:00  3220.84  3219.32  3219.840  1639.0  ...  520.120   
2021-01-05 15:57:00-05:00  3222.70  3219.60  3222.700  1951.0  ...  519.850   
2021-01-05 15:58:00-05:00  3221.79  3221.18  3221.180  1112.0  ...  520.430   
2021-01-05 15:59:00-05:00  3220.38  3219.03  3219.670  3164.0  ...  520.360   
2021-01-05 16:00:00-05:00      NaN      NaN       NaN     NaN  ...      NaN   

                                                               TSLA           \
                              high     low    close  volume    open     high   
2021-01-05 09:30:00-05:00  521.980  520.77  521.030  1355.0  723.66  726.280   
2021-01-05 09:31:00-05:00  521.755  520.92  521.365  1112.0  726.60  726.999   
2021-01-05 09:32:00-05:00  522.355  520.77  520.770  1347.0  723.10  723.100   
2021-01-05 09:33:00-05:00  520.840  520.00  520.000  1582.0  720.53  722.710   
2021-01-05 09:34:00-05:00  522.260  521.37  522.240  1039.0  719.97  724.220   
...                            ...     ...      ...     ...     ...      ...   
2021-01-05 15:56:00-05:00  520.150  519.57  519.570  3542.0  732.89  733.490   
2021-01-05 15:57:00-05:00  520.460  519.76  520.460  1527.0  733.05  734.585   
2021-01-05 15:58:00-05:00  520.650  520.29  520.300  1804.0  734.28  734.830   
2021-01-05 15:59:00-05:00  520.890  520.24  520.760  7235.0  734.63  735.420   
2021-01-05 16:00:00-05:00      NaN     NaN      NaN     NaN     NaN      NaN   

                                                    
                              low   close   volume  
2021-01-05 09:30:00-05:00  721.35  725.23  18284.0  
2021-01-05 09:31:00-05:00  722.42  723.00   7760.0  
2021-01-05 09:32:00-05:00  719.78  720.57   9902.0  
2021-01-05 09:33:00-05:00  719.22  719.71   7086.0  
2021-01-05 09:34:00-05:00  719.97  724.22   8581.0  
...                           ...     ...      ...  
2021-01-05 15:56:00-05:00  732.78  733.00   4909.0  
2021-01-05 15:57:00-05:00  733.05  734.49   7281.0  
2021-01-05 15:58:00-05:00  734.09  734.83  11767.0  
2021-01-05 15:59:00-05:00  734.62  735.33  13446.0  
2021-01-05 16:00:00-05:00     NaN     NaN      NaN  

[391 rows x 35 columns]

#### 3. Store only the close prices from the `prices` DataFrame in a new DataFrame called `df_closing_prices`, then view the head and tail to confirm the following:
* First price for each stock on the open at 9:30 Eastern Time.
* Last price for the day on the close at 3:59 pm Eastern Time.

In [9]:
# Fetch the closing prices for each one of the tickers and store in a column in df_closing_prices named after that ticker
df_closing_prices = pd.DataFrame(data={ticker: prices[ticker]['close'] for ticker in tickers}, columns=tickers)

In [10]:
# Preview first five rows
df_closing_prices.head(5)

,FB,AMZN,AAPL,NFLX,GOOGL,MSFT,TSLA
2021-01-05 09:30:00-05:00,269.00,3172.980,129.485,521.030,1724.17,217.65,725.23
2021-01-05 09:31:00-05:00,269.17,3177.810,130.060,521.365,1724.05,217.63,723.00
2021-01-05 09:32:00-05:00,269.72,3175.470,130.020,520.770,1721.61,217.77,720.57
2021-01-05 09:33:00-05:00,268.80,3179.360,130.120,520.000,NaN,217.72,719.71
2021-01-05 09:34:00-05:00,269.58,3184.015,130.510,522.240,1720.30,217.31,724.22


In [11]:
# Preview last five rows
df_closing_prices.tail(5)

,FB,AMZN,AAPL,NFLX,GOOGL,MSFT,TSLA
2021-01-05 15:56:00-05:00,270.65,3219.84,130.850,519.57,1738.15,217.970,733.00
2021-01-05 15:57:00-05:00,270.91,3222.70,131.010,520.46,1738.99,218.175,734.49
2021-01-05 15:58:00-05:00,270.88,3221.18,130.990,520.30,1738.84,218.150,734.83
2021-01-05 15:59:00-05:00,270.86,3219.67,130.965,520.76,1740.57,218.000,735.33
2021-01-05 16:00:00-05:00,NaN,NaN,131.140,NaN,NaN,NaN,NaN


#### 4. When viewing the head and tail, you'll notice several `NaN` values.
* Alpaca reports `NaN` for minutes without any trades occuring as missing.
* These values must be removed, we use Panda's `ffill()` function to "forward fill", or replace, those prices with the previous values (since the price has not changed).


In [12]:
# Use Pandas' forward fill function to fill missing values (be sure to set inplace=True)
df_closing_prices.ffill(inplace=True)

### Computing Returns

#### 1. Compute the percentage change values for 1 minute as follows:
* Create a variable called `forecast` to hold the forecast, in this case `1` for 1 minute.
* Use the `pct_change` function, passing in the `forecast`, on the `df_closing_prices` DataFrame, storeing the newly generated DataFrame in a variable called `returns`.
* Convert the `returns` DataFrame to show forward returns by passing `-(forecast)` into the `shift function.`

In [13]:
# Define a variable to set prediction period
forecast = 1
# Compute the pct_change for 1 min 
returns = df_closing_prices.pct_change(forecast)
# Shift the returns to convert them to forward returns
returns = returns.shift(-(forecast))
# Preview the DataFrame
returns.head(5)

,FB,AMZN,AAPL,NFLX,GOOGL,MSFT,TSLA
2021-01-05 09:30:00-05:00,0.000632,0.001522,0.004441,0.000643,-0.000070,-0.000092,-0.003075
2021-01-05 09:31:00-05:00,0.002043,-0.000736,-0.000308,-0.001141,-0.001415,0.000643,-0.003361
2021-01-05 09:32:00-05:00,-0.003411,0.001225,0.000769,-0.001479,0.000000,-0.000230,-0.001193
2021-01-05 09:33:00-05:00,0.002902,0.001464,0.002997,0.004308,-0.000761,-0.001883,0.006266
2021-01-05 09:34:00-05:00,0.001335,0.002074,0.000651,-0.001704,0.003061,-0.001703,0.004667


##### Note: 
> You can verify these returns are computed correctly by analyzing the first observation for Facebook:
> * 9:30 am for 0.000632.
 
> How is that number computed? 
 
> * The price of Facebook at 9:30 is 269.00
> * The price of Facebook at 9:31 is 269.17

> Which gives you:

> * (269.17 - 	269.00)/ 269.90 = 0.000632
 

#### 2. Convert the DataFrame into long form for merging later using `unstack` and `reset_index`.

In [14]:
# Use unstack() to bring the data in long format and save the output as as dataframe
returns = returns.unstack().to_frame()
# Rename the column to make it easer to identify it:
name = f'F_{forecast}_m_returns'
returns.rename(columns={0: name}, inplace=True)
# Reset the index of the dataframe for merging later (be sure to set inplace=True)
returns.reset_index(inplace=True)

In [15]:
# Preview the first five rows
returns.head(5)

,level_0,level_1,F_1_m_returns
0,FB,2021-01-05 09:30:00-05:00,0.000632
1,FB,2021-01-05 09:31:00-05:00,0.002043
2,FB,2021-01-05 09:32:00-05:00,-0.003411
3,FB,2021-01-05 09:33:00-05:00,0.002902
4,FB,2021-01-05 09:34:00-05:00,0.001335


In [16]:
# Preview the last five rows
returns.tail(5)

,level_0,level_1,F_1_m_returns
2732,TSLA,2021-01-05 15:56:00-05:00,0.002033
2733,TSLA,2021-01-05 15:57:00-05:00,0.000463
2734,TSLA,2021-01-05 15:58:00-05:00,0.000680
2735,TSLA,2021-01-05 15:59:00-05:00,0.000000
2736,TSLA,2021-01-05 16:00:00-05:00,NaN


#### 3. Compute the 1, 5, 10 minute momentums that will be used to predict the forward returns, then merge them with the forward returns as follows:
* Create the list of moments: `list_of_momentums = [1,5,10]`.
* Write a for-loop to loop through the `list_of_momentums`, applying them to `pct_change` with the `df_closing_price` with each iteration.
* With each loop, the data temporary DataFrame, `returns_temp` will need to be prepped with `unstack` and `reset_index`, then added as a new column to the original `returns` DataFrame from the prior step.
* Complete this step by dropping the null values from `returns` and creating a multi-index based on date and ticker.

In [17]:
# Create list of momentums that we want to predict
list_of_momentums = [1,5,10]
for i in list_of_momentums:   
    # Compute percentage change for each one of the momentums in the momentum list
    returns_temp = df_closing_prices.pct_change()
    # Unstack the returns and save the output as as dataframe called returns_temp 
    returns_temp = returns_temp.unstack().to_frame()
    # Rename the column to make it easer to identify it:
    returns_temp = returns_temp.rename(columns={0: f'{i}_m_returns'})
    # Reset the index so we can merge based on index
    returns_temp = returns_temp.reset_index()
    # Merge returns_temp  with the original returns 
    returns = pd.merge(returns,returns_temp,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))

In [18]:
returns.head(11)

,level_0,level_1,F_1_m_returns,1_m_returns,5_m_returns,10_m_returns
0,FB,2021-01-05 09:30:00-05:00,0.000632,NaN,NaN,NaN
1,FB,2021-01-05 09:31:00-05:00,0.002043,0.000632,0.000632,0.000632
2,FB,2021-01-05 09:32:00-05:00,-0.003411,0.002043,0.002043,0.002043
3,FB,2021-01-05 09:33:00-05:00,0.002902,-0.003411,-0.003411,-0.003411
4,FB,2021-01-05 09:34:00-05:00,0.001335,0.002902,0.002902,0.002902
5,FB,2021-01-05 09:35:00-05:00,0.000185,0.001335,0.001335,0.001335
6,FB,2021-01-05 09:36:00-05:00,0.000778,0.000185,0.000185,0.000185
7,FB,2021-01-05 09:37:00-05:00,-0.000777,0.000778,0.000778,0.000778
8,FB,2021-01-05 09:38:00-05:00,0.001000,-0.000777,-0.000777,-0.000777
9,FB,2021-01-05 09:39:00-05:00,0.000074,0.001000,0.001000,0.001000


In [19]:
# Use dropna() to get rid of those missing observations.
returns = returns.dropna()
# Rename columns level_0 and level_1
returns = returns.rename(columns={'level_0': 'ticker', 'level_1': 'time'})
# Create a multi index based on ticker and time
returns = returns.set_index(['ticker', 'time'], inplace=False)

In [20]:
returns

F_1_m_returns  1_m_returns  5_m_returns  \
ticker time                                                                 
FB     2021-01-05 09:31:00-05:00       0.002043     0.000632     0.000632   
       2021-01-05 09:32:00-05:00      -0.003411     0.002043     0.002043   
       2021-01-05 09:33:00-05:00       0.002902    -0.003411    -0.003411   
       2021-01-05 09:34:00-05:00       0.001335     0.002902     0.002902   
       2021-01-05 09:35:00-05:00       0.000185     0.001335     0.001335   
...                                         ...          ...          ...   
TSLA   2021-01-05 15:55:00-05:00      -0.000259     0.000437     0.000437   
       2021-01-05 15:56:00-05:00       0.002033    -0.000259    -0.000259   
       2021-01-05 15:57:00-05:00       0.000463     0.002033     0.002033   
       2021-01-05 15:58:00-05:00       0.000680     0.000463     0.000463   
       2021-01-05 15:59:00-05:00       0.000000     0.000680     0.000680   

                                  10_m_returns  
ticker time                                     
FB     2021-01-05 09:31:00-05:00      0.000632  
       2021-01-05 09:32:00-05:00      0.002043  
       2021-01-05 09:33:00-05:00     -0.003411  
       2021-01-05 09:34:00-05:00      0.002902  
       2021-01-05 09:35:00-05:00      0.001335  
...                                        ...  
TSLA   2021-01-05 15:55:00-05:00      0.000437  
       2021-01-05 15:56:00-05:00     -0.000259  
       2021-01-05 15:57:00-05:00      0.002033  
       2021-01-05 15:58:00-05:00      0.000463  
       2021-01-05 15:59:00-05:00      0.000680  

[2723 rows x 4 columns]

## Part 2: Train and Compare Multiple Machine Learning Algorithms

 In this section, you'll train each of the requested algorithms and compare performance. Be sure to use the same parameters and training steps for each model. This is necessary to compare each model accurately.

### Preprocessing Data

#### 1. Generate your feature data (`X`) and target data (`y`):
* Create a dataframe `X` that contains all the columns from the returns dataframe that will be used to predict `F_1_m_returns`.
* Create a variable, called `y`, that is equal 1 if `F_1_m_returns` is larger than 0. This will be our target variable.

In [21]:
# Load the dataset returns.csv and set the index to level_0 and time
dummy_returns = pd.read_csv(Path('../Data/returns.csv')) 

In [22]:
# Create a separate dataframe for features and define the target variable as a binary target
X = returns.drop(columns=['F_1_m_returns'])

# Create the target variable
y = np.where((returns['F_1_m_returns'] > 0), 1, 0)

X, y

(                                  1_m_returns  5_m_returns  10_m_returns
 ticker time                                                             
 FB     2021-01-05 09:31:00-05:00     0.000632     0.000632      0.000632
        2021-01-05 09:32:00-05:00     0.002043     0.002043      0.002043
        2021-01-05 09:33:00-05:00    -0.003411    -0.003411     -0.003411
        2021-01-05 09:34:00-05:00     0.002902     0.002902      0.002902
        2021-01-05 09:35:00-05:00     0.001335     0.001335      0.001335
 ...                                       ...          ...           ...
 TSLA   2021-01-05 15:55:00-05:00     0.000437     0.000437      0.000437
        2021-01-05 15:56:00-05:00    -0.000259    -0.000259     -0.000259
        2021-01-05 15:57:00-05:00     0.002033     0.002033      0.002033
        2021-01-05 15:58:00-05:00     0.000463     0.000463      0.000463
        2021-01-05 15:59:00-05:00     0.000680     0.000680      0.000680
 
 [2723 rows x 3 columns],
 array([1,

##### Note:
> Notice that we don't use shuffle when splitting the dataset into a training and testing dataset. 

> We want to keep the original ordering of the data, so we don't end up using observations in the future to predict past observations,

> This is a critical mistake known as look ahead bias.

#### 2. Use the train_test_split library to split the dataset into a training and testing dataset, with 70% used for testing
* Set the shuffle parameter to False, so that you use the first 70% for training to prvent look ahead bias.
* Make sure you have these 4 variables: `X_train`, `X_test`, `y_train`, `y_test`. 

In [23]:
# Import train_test_split 
from sklearn.model_selection import train_test_split

# Split the dataset without shuffling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

#### 3. Use the `Counter` function to test the distribution of the data. 
* The result of `Counter({1: 668, 0: 1194})` reveals the data is indeed unbalanced.

In [24]:
# Import the Counter function from the collections library
from collections import Counter

# Use Counter to count the number 1s and 0 in y_train
Counter(y_train)

Counter({1: 695, 0: 1211})

#### 4. Balance the dataset with the Oversampler libary, setting `random state= 1`.

In [25]:
# Import RandomOverSampler from the imblearn library
from imblearn.over_sampling import RandomOverSampler

# Use RandomOverSampler to resample the datase using random_state=1
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#### 5. Test the distribution once again with `Counter`. The new result of `Counter({1: 1194, 0: 1194})` shows the data is now balanced.

In [26]:
# Use Counter again to verify imbalance removed
Counter(y_resampled)

Counter({1: 1211, 0: 1211})

# Machine Learning

#### 1. The first cells in this section provide an example of how to fit and train your model using the `LogisticRegression` model from sklearn:
* Import select model.
* Instantiate model object.
* Fit the model to the resampled data - `X_resampled` and `y_resampled`.
* Predict the model using `X_test`.
* Print the classification report.

In [27]:
# Import classification_report from sklearn
from sklearn.metrics import classification_report

In [28]:
# Import LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression

# Create a LogisticRegression model and train it on the X_resampled data we created before
log_model = LogisticRegression()
log_model.fit(X_resampled, y_resampled)  

# Use the model you trained to predict using X_test
y_pred = log_model.predict(X_test)   

# Print out a classification report toevaluate performance
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.4938    0.4771    0.4853       415
           1     0.4784    0.4950    0.4866       402

    accuracy                         0.4859       817
   macro avg     0.4861    0.4861    0.4859       817
weighted avg     0.4862    0.4859    0.4859       817



#### 2. Use the same approach as above to train and test the following ML Algorithms:
* [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)
* [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)
* [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

#### RandomForestClassifier

In [29]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Create a RandomForestClassifier model and train it on the X_resampled data we created before
rfc_model = RandomForestClassifier().fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
rfc_predictions = rfc_model.predict(X_test)

# Print out a classification report to evaluate performance
print(classification_report(y_test, rfc_predictions))

              precision    recall  f1-score   support

           0       0.51      0.67      0.58       415
           1       0.50      0.33      0.40       402

    accuracy                           0.51       817
   macro avg       0.50      0.50      0.49       817
weighted avg       0.50      0.51      0.49       817



#### GradientBoostingClassifier

In [30]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import GradientBoostingClassifier

# Create a GradientBoostingClassifier model and train it on the X_resampled data we created before
gbc_model = GradientBoostingClassifier().fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
gbc_predictions = gbc_model.predict(X_test)

# Print out a classification report to evaluate performance
print(classification_report(y_test, gbc_predictions))

              precision    recall  f1-score   support

           0       0.50      0.36      0.42       415
           1       0.49      0.63      0.55       402

    accuracy                           0.49       817
   macro avg       0.49      0.49      0.48       817
weighted avg       0.49      0.49      0.48       817



#### AdaBoostClassifier

In [31]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import AdaBoostClassifier

# Create a AdaBoostClassifier model and train it on the X_resampled data we created before
abc_model = AdaBoostClassifier().fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
abc_predictions = abc_model.predict(X_test)

# Print out a classification report to evaluate performance
print(classification_report(y_test, abc_predictions))

              precision    recall  f1-score   support

           0       0.44      0.16      0.24       415
           1       0.48      0.79      0.59       402

    accuracy                           0.47       817
   macro avg       0.46      0.48      0.42       817
weighted avg       0.46      0.47      0.41       817



#### XGBClassifier

In [32]:
# Import RandomForestClassifier from sklearn
from xgboost import XGBClassifier

# Create a XGBClassifier model and train it on the X_resampled data we created before
xbgc_model = XGBClassifier().fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
xbgc_predictions = xbgc_model.predict(X_test)

# Print out a classification report to evaluate performance
print(classification_report(y_test, xbgc_predictions))

[07:38:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.48      0.56      0.52       415
           1       0.46      0.38      0.42       402

    accuracy                           0.47       817
   macro avg       0.47      0.47      0.47       817
weighted avg       0.47      0.47      0.47       817



C:\Users\ayres\miniconda3\envs\machineLearning\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


### Evaluate the performance of each model


#### 1. Using the classification report for each model, choose the model with the highest precision for use in your algo-trading program.
#### 2. Save the selected model with the `joblib` libary to avoid retraining every time you wish to use it.

In [33]:
# Import the joblib library 
import joblib

# Use the library to save the model that you want to use for trading
joblib.dump(log_model, Path('../Config/log_model.pkl'))

['..\\Config\\log_model.pkl']

## Part 3: Implement the strongest model using Apaca API

### Develop the Algorithm


#### 1. Use the provided code to ping the Alpaca API and create the DataFrame needed to feed data into the model.
   * This code will also store the correct feature data in `X` for later use.

In [34]:
# Create the list of tickers

ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
# Define Dates

beg_date = '2021-01-06'
end_date = '2021-01-06'

# Convert the date in a format the Alpaca API reqires
start =  pd.Timestamp(f'{beg_date} 09:30:00-0400', tz='America/New_York').replace(hour=9, minute=30, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
end   =  pd.Timestamp(f'{end_date} 16:00:00-0400', tz='America/New_York').replace(hour=15, minute=0, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
timeframe='1Min'

# Use iloc to get the last 10 mins every time we pull new data
prices = alpaca.get_barset(ticker_list, "minute", start=start, end=end).df.iloc[-11:]
prices.ffill(inplace=True)   

# Create an empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices of our tickers
df_closing_prices["FB"] = prices["FB"]["close"]
df_closing_prices["AMZN"] = prices["AMZN"]["close"]
df_closing_prices["AAPL"] = prices["AAPL"]["close"]
df_closing_prices["NFLX"] = prices["NFLX"]["close"]
df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
df_closing_prices['MSFT'] = prices['MSFT']["close"]
df_closing_prices['TSLA'] = prices['TSLA']["close"]

print(df_closing_prices.head(20))

                                FB      AMZN     AAPL    NFLX    GOOGL  \
2021-01-06 14:50:00-05:00  264.610  3146.960  127.110  506.54  1721.82   
2021-01-06 14:51:00-05:00  264.630  3146.910  127.430  506.54  1721.82   
2021-01-06 14:52:00-05:00  264.830  3147.980  127.720  506.69  1723.67   
2021-01-06 14:53:00-05:00  264.525  3148.570  127.510  506.01  1723.67   
2021-01-06 14:54:00-05:00  264.560  3147.840  127.645  506.01  1720.84   
2021-01-06 14:55:00-05:00  264.880  3150.330  127.920  506.30  1720.60   
2021-01-06 14:56:00-05:00  264.965  3150.610  128.150  506.72  1721.10   
2021-01-06 14:57:00-05:00  264.980  3151.745  127.980  507.07  1720.07   
2021-01-06 14:58:00-05:00  265.000  3149.280  127.850  506.33  1720.07   
2021-01-06 14:59:00-05:00  265.360  3150.840  127.930  506.13  1720.48   
2021-01-06 15:00:00-05:00  264.840  3148.580  127.630  506.43  1720.48   

                              MSFT    TSLA  
2021-01-06 14:50:00-05:00  213.930  753.33  
2021-01-06 14:51:00-0

In [35]:
# Create list of momentums
list_of_momentums = [1,5,10]

for i in list_of_momentums:  
    # Compute percentage change for each one of the momentums in the momentum list
    returns_temp = df_closing_prices.pct_change(i)
    # Unstack the returns 
    returns_temp = pd.DataFrame(returns_temp.unstack())
    name = f'{i}_m_returns'
    returns_temp.rename(columns={0: name}, inplace = True)
    # Reset the index so we can merge based on index
    returns_temp.reset_index(inplace = True)
    # Merge newly computed returns with previously created returns
    if i == 1:
        returns = returns_temp
    else:
        returns = pd.merge(returns,returns_temp,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))

# Drop nulls and set index
returns.dropna(axis=0, how='any', inplace=True)
returns.set_index(['level_0', 'level_1'], inplace=True)

# Generate feature data and preview first 10 rows.
X = returns
X.head(10)

,,1_m_returns,5_m_returns,10_m_returns
level_0,level_1,,,
FB,2021-01-06 15:00:00-05:00,-0.001960,-0.000151,0.000869
AMZN,2021-01-06 15:00:00-05:00,-0.000717,-0.000555,0.000515
AAPL,2021-01-06 15:00:00-05:00,-0.002345,-0.002267,0.004091
NFLX,2021-01-06 15:00:00-05:00,0.000593,0.000257,-0.000217
GOOGL,2021-01-06 15:00:00-05:00,0.000000,-0.000070,-0.000778
MSFT,2021-01-06 15:00:00-05:00,-0.000770,-0.001213,0.000654
TSLA,2021-01-06 15:00:00-05:00,-0.000735,-0.001587,0.010500


#### 2. Using `joblib`, load the chosen model.

In [36]:
# Load the previously trained and saved model using joblib
loaded_log_model = joblib.load(Path('../Config/log_model.pkl'))

#### 3. Use the model file to make predicttions:
* Use `predict` on `X` and save this as `y_pred`.
* Convert `y_pred` to a DataFrame, setting the index to the index of `X`.
* Rename the column 0 to 'buy', be sure to set `inplace =True`.

In [37]:
# Use the model file to predict on X
y_pred = loaded_log_model.predict(X)

# Convert y_pred to a dataframe, set the index to the index of X
y_pred_df = pd.DataFrame(data=y_pred, index=X.index)

# Rename the column 0 to 'buy', be sure to set inplace =True
y_pred_df = y_pred_df.rename(columns={0: 'buy'})

y_pred_df

,,buy
level_0,level_1,
FB,2021-01-06 15:00:00-05:00,1
AMZN,2021-01-06 15:00:00-05:00,1
AAPL,2021-01-06 15:00:00-05:00,1
NFLX,2021-01-06 15:00:00-05:00,0
GOOGL,2021-01-06 15:00:00-05:00,1
MSFT,2021-01-06 15:00:00-05:00,1
TSLA,2021-01-06 15:00:00-05:00,0


#### 4. Filter the stocks where 'buy' is equal to 1, saving the filter as `y_pred`.

In [38]:
# Filter the stocks where 'buy' is equal to 1
y_pred = y_pred_df.loc[(y_pred_df['buy'] == 1)]
y_pred

,,buy
level_0,level_1,
FB,2021-01-06 15:00:00-05:00,1
AMZN,2021-01-06 15:00:00-05:00,1
AAPL,2021-01-06 15:00:00-05:00,1
GOOGL,2021-01-06 15:00:00-05:00,1
MSFT,2021-01-06 15:00:00-05:00,1


#### 5. Using the `y_pred` filter, create a dictionary called `buy_dict` and assign 'n' to each Ticker (key value) as a placeholder.

In [39]:
# Create dictionary from y_pred and assign a 'n' to each of them for now as a placeholder.
buy_dict = dict.fromkeys(y_pred.index.get_level_values(0), 'n')
buy_dict

{'FB': 'n', 'AMZN': 'n', 'AAPL': 'n', 'GOOGL': 'n', 'MSFT': 'n'}

#### 6. Obtain the total available equity in your account from the Alpaca API and store in a variable called `total_capital`. You will split the capital equally between all selected stocks per the CIO's request.

In [40]:
# Pull the total available equity in our account from the  Alpaca API
total_capital = alpaca.get_account().equity
total_capital

'100000'

In [41]:
# Compute capital per stock, divide equity in account by number of stocks
# Use Alpaca API to pull the equity in the account
if len(buy_dict) > 0:
    capital_per_stock = float(total_capital)/ len(buy_dict)
else:
    capital_per_stock = 0
print(f'Capital per stock: {capital_per_stock}')

Capital per stock: 20000.0


#### 7. Use a for-loop to iterate through `buy_dict` to determine the number stocks you need to buy for each ticker.

In [42]:
# Use for loop to iterate through dictionary of buys 
# Determine the number stocks we need to buy for each ticker
for ticker in buy_dict:
    try:
        buy_dict[ticker] = int(capital_per_stock /int(prices[ticker].iloc[-1]['close']))
    except:
        pass

print(buy_dict)

{'FB': 75, 'AMZN': 6, 'AAPL': 157, 'GOOGL': 11, 'MSFT': 93}


#### 8. Cancel all previous orders in the Alpaca API (so you don't buy more than intended) and sell all currently held stocks to close all positions.

In [43]:
# Cancel all previous orders in the Alpaca API
alpaca.cancel_all_orders()

# Sell all currently held stocks to close all positions
alpaca.close_all_positions()

[]

#### 9. Iterate through `buy_dict` and send a buy order for each ticker with their corresponding number of shares.

In [44]:
# Iterate through the longlist object and send a buy order for each ticker with a corresponding number of shares:
for ticker, amount_shares in buy_dict.items():
    alpaca.submit_order(
        symbol=ticker,
        qty=amount_shares,
        side='buy',
        time_in_force='gtc',
        type='limit',
        limit_price=df_closing_prices[ticker].iloc[-1]
    )

### Automate the algorithm

#### 1. Make a function called `trade()` that incorporates all of the steps above.

In [45]:
# Add all of the steps conducted above into the function trade
def trade():

    ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
    # Notice that we remove the start and end variables since we want the latest prices.
    timeframe='1Min'
    # Use iloc to get the last 10 mins every time we pull new data
    prices = alpaca.get_barset(ticker_list, timeframe).df.iloc[:-11]
    prices.ffill(inplace=True)   

    # Create an empty DataFrame for closing prices
    df_closing_prices = pd.DataFrame()

    # Fetch the closing prices of our tickers
    df_closing_prices["FB"] = prices["FB"]["close"]
    df_closing_prices["AMZN"] = prices["AMZN"]["close"]
    df_closing_prices["AAPL"] = prices["AAPL"]["close"]
    df_closing_prices["NFLX"] = prices["NFLX"]["close"]
    df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
    df_closing_prices['MSFT'] = prices['MSFT']["close"]
    df_closing_prices['TSLA'] = prices['TSLA']["close"]

    # Initialize returns DataFrame with forecasted returns
    returns = df_closing_prices.pct_change(1).shift(-1).unstack().to_frame().rename(columns={0: 'F_1_m_returns'}).reset_index()
    
    # Loop through momentums to build new DataFrame
    list_of_momentums = [1,5,10]
    for i in list_of_momentums:   
        returns_temp = df_closing_prices.pct_change(i)
        returns_temp = pd.DataFrame(returns_temp.unstack())
        name = f'{i}_m_returns'
        returns_temp.rename(columns={0: name}, inplace = True)
        returns_temp.reset_index(inplace = True)
        returns = pd.merge(returns,returns_temp,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))

    # Drop nulls and set index            
    returns.dropna(axis=0, how='any', inplace=True)
    returns = returns.rename(columns={'level_0': 'ticker', 'level_1': 'time'})
    returns.set_index(['ticker', 'time'], inplace=True)

    # Preprocess data for model
    X = returns
    y = np.where((returns['F_1_m_returns'] > 0), 1, 0)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

    ros_model = RandomOverSampler(random_state=1)
    X_train_resamp, y_train_resamp = ros_model.fit_resample(X_train, y_train)

    rfc_model = RandomForestClassifier().fit(X_train_resamp, y_train_resamp)
    rfc_predictions = rfc_model.predict(X)
    rfc_predictions_df = pd.DataFrame(data=rfc_predictions, index=X.index)
    rfc_predictions_df = rfc_predictions_df.rename(columns={0: 'side'})
    
    # Create the `buy_dict` object
    buy_dict = {ticker: 'n' for ticker in rfc_predictions_df.index.get_level_values(0)}

    # Split capital between stocks and determine buy or sell
    total_capital = alpaca.get_account().equity

    if len(buy_dict) > 0:
        capital_per_stock = float(total_capital)/ len(buy_dict)
    else:
        capital_per_stock = 0
    print(f'Capital per stock: {capital_per_stock}')

    for ticker in buy_dict:
        try:
            buy_dict[ticker] = int(capital_per_stock /int(prices[ticker].iloc[-1]['close']))
        except:
            pass

    
    # Cancel pending orders and close positions
    alpaca.cancel_all_orders()
    alpaca.close_all_positions()
    
    time.sleep(1)

    # Check if currently holding asset or have an open order and submit orders accordingly
    for ticker, amount_shares in buy_dict.items():
        alpaca.submit_order(
            symbol=ticker,
            qty=amount_shares,
            side=('buy' if (rfc_predictions_df.loc[ticker].iloc[-1]['side'] == 1) else 'sell'),
            time_in_force='gtc',
            type='limit',
            limit_price=df_closing_prices[ticker].iloc[-1]
        )


In [46]:
trade()

Capital per stock: 14285.714285714286


#### 2. Import Python's schedule module.

In [47]:
# Import Python's schedule module 
import schedule

#### 3. Use the "schedule" module to automate the algorithm:
* Clear the schedule with `.clear()`.
* Define a schedule to run the trade function every minute at 5 seconds past the minute mark (e.g. `10:31:05`).
* Use the Alpaca API to check whether the market is open.
* Use run_pending() function inside schedule to execute the schedule you defined while the market is open

In [54]:
# Clear the schedule
schedule.clear()

# Define a schedule to run the trade function every minute at 5 seconds past the minute mark (e.g. 10:31:05)
schedule.every().minute.at(':05').do(trade)

# Use the Alpaca API to check whether the market is open
alpaca.get_clock().is_open

# Use run_pending() function inside schedule to execute the schedule you defined as long as the market is open
while True:
    while alpaca.get_clock().is_open:
        schedule.run_pending()
        time.sleep(1)

sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/clock 3 more time(s)...
sleep 3 seconds and r

Capital per stock: 14285.714285714286


APIError: cannot open a long buy while a short sell order is open